In [1]:
from itertools import *
from typing import *

import numpy as np
import pandas as pd

In [2]:
from pyqubo import Binary

In [3]:
nodes = pd.read_csv("task-2-nodes.csv", header=None)

all_places = []
good_places = {2: [], 3: [], 5: [], 9: []}
all_good_places = []

for i, node in enumerate(nodes.iloc):
    name = node[0]
    value = node[1]
    # print(name, value)

    all_places += [name]
    if value != 0:
        all_good_places += [i]
        good_places[value] += [i]

adjacency_matrix = pd.read_csv("task-2-adjacency_matrix.csv", index_col=0)

edges = {}
not_edges = {}
for p0, place in enumerate(all_places):
    # print(adjacency_matrix[p0])
    for p1, value in enumerate(adjacency_matrix[place]):
        # print(p1)
        if value == "-":
            not_edges[(p0, p1)] = True
        elif p0 != p1:
            edges[(p0, p1)] = int(value)

In [4]:
T = 15
X = len(all_places)
K = 15

# buses[k,x,t] – Автобус #k находится в месте #x в момент времени #t
buses = np.array([
    [
        [
            Binary(f"buses[{k},{x},{t}]")
            for t in range(T + 1)
        ]
        for x in range(X)
    ]
    for k in range(K)
])

# buses_per_edge[u,v] - Количество автобусов едущих в любую сторону по ребру u<->v
buses_per_edge = np.array([
    [
        Binary(f"buses_per_edge[{u},{v}]")
        for u in range(X)
    ]
    for v in range(X)
])

# visit[k,x] - Автобус #k посетил `хорошее` место #x за всю свою поездку
visit = np.array([
    [
        Binary(f"visit[{k},{x}]")
        for x in range(X)
    ]
    for k in range(K)
])

# passengers[k,p] - Автобус #k везет == p пассажиров
passengers = np.array([
    [
        Binary(f"passengers[{k},{p}]")
        for p in [2, 3, 5, 9]
    ]
    for k in range(K)
])

In [5]:
np.prod(buses.shape) + np.prod(buses_per_edge.shape) + np.prod(visit.shape) + np.prod(passengers.shape)

np.int64(17844)

In [6]:
# Каждый автобус в каждый момент находится только в одной точке
H0 = np.sum((np.sum(buses, axis=1) - 1) ** 2)

# Каждый автобус в каждом `хорошем` месте побывает <= 1 раза за маршрут
# H1 = np.sum(buses, axis=2)[:,]
# H1 = np.sum((np.sum(buses, axis=2)[:,good_places] - visit) ** 2)
H1 = np.sum((np.sum(buses[:,all_good_places,:], axis=2) - visit[:,all_good_places]) ** 2)

# В каждом месте в каждое время находится только один автобус
H2 = np.sum((np.sum(buses, axis=0) - 1) ** 2)

# Каждый автобус везет одно количество пассажиров
H3 = np.sum((np.sum(passengers, axis=1) - 1) ** 2)

# Начало и конец на вокзале
ts = all_places.index("Вокзал")
H4 = np.sum((buses[:,ts,0] - 1) ** 2) + np.sum((buses[:,ts,-1] - 1) ** 2)

In [7]:
# Маршрут не может проходить через дорогу, которой нет
H5 = np.sum([
    np.sum(buses[:,u,:-1] * buses[:,v,1:])
    for u, v in not_edges if v > u
])

Максимизируем чистую прибыль всех автобусов:
- Доход - сумма всех билетов с рейса
- Расход - проезд по ребрам с весом (бензин и тд.)

Доход с одного автобуса равен:
$$
\text{Income} = N * \sum_{n\ge N} S(n)
$$
Где $N$ - количество пассажиров в автобусе, $S(N)$ - количество мест на маршруте, где можно уместить ровно $N$ пассажиров.
Рассмотрим 4 случая разного количества пассажиров:
$$
\begin{matrix}
N = 2 & \Rightarrow & \text{Income} = 2 * \left(S(2) +  S(3) +  S(5) +  S(9)\right) \\
N = 3 & \Rightarrow & \text{Income} = 3 * \left(        S(3) +  S(5) +  S(9)\right) \\
N = 5 & \Rightarrow & \text{Income} = 5 * \left(                S(5) +  S(9)\right) \\
N = 9 & \Rightarrow & \text{Income} = 9 * \left(                        S(9)\right)
\end{matrix}
$$

Расход с одного автобуса равен:
$$
\text{Outcome} = \sum_{(u,v)\in \left(\text{edges}\wedge\text{bus route}\right)} \text{cost}_{u,v}
$$

Вместо максимизации $\text{Income}$, минимизируем значение $\overline{\text{Income}} = \text{Max Income} - \text{Income}$. Таким образом, значение $\overline{\text{Income}}$ тоже лежит в промежутке $[0, \text{Max Income}]$. 

In [9]:
_buses = np.sum(buses, axis=2)

P_MAX = 9 * (T - 1)
W_MAX = max(edges.values()) * T

COST = P_MAX - (
    np.sum(2 * passengers[:,0] * np.sum(_buses[:,   good_places[2] +    good_places[3] +    good_places[5] +    good_places[9]], axis=1)) +
    np.sum(3 * passengers[:,1] * np.sum(_buses[:,                       good_places[3] +    good_places[5] +    good_places[9]], axis=1)) +
    np.sum(5 * passengers[:,2] * np.sum(_buses[:,                                           good_places[5] +    good_places[9]], axis=1)) +
    np.sum(9 * passengers[:,3] * np.sum(_buses[:,                                                               good_places[9]], axis=1)) +
    0
) + (
    np.sum([
        edges[(u, v)] * buses[k,u,t] * buses[k,v,t+1]
        for t in range(T)
        for u, v in edges
        for k in range(K)
    ])
)

In [10]:
BQM = H0 + H1 + H2 + H3 + H4 + H5 + COST / (P_MAX + W_MAX)

In [17]:
model = BQM.compile()
qubo, offset = model.to_qubo(index_label=True)

In [20]:
variables = len(model.variables)
qubo_list = sorted([(r, c, int(v)) for (r, c), v in qubo.items() if int(v) != 0])

with open("Task_2.edges", "w") as f:
    f.write(f"{variables} {len(qubo_list)}\n")
    for r, c, v in qubo_list:
        f.write(f"{r} {c} {v}\n")